# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather = pd.read_csv('../Output/City_WeatherData.csv', encoding='utf-8')
city_weather.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,san patricio,28.0170,-97.5169,69.01,100.0,90.0,4.61,US,1.612957e+09
1,takoradi,4.8845,-1.7554,86.56,67.0,11.0,7.29,GH,1.612957e+09
2,svetlogorsk,52.6333,29.7333,15.12,96.0,99.0,8.59,BY,1.612957e+09
3,albany,42.6001,-73.9662,19.40,86.0,90.0,8.05,US,1.612957e+09
4,airai,-8.9266,125.4092,62.98,100.0,99.0,1.97,TL,1.612957e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# # Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' as locations
locations = city_weather[['Lat', 'Lng']]

# Convert Humidity to float and store 
humidity = city_weather['Humidity'].astype(float)
max_humidity = humidity.max()


In [4]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity= max_humidity,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with max temp between 60 and 80, Wind Speed less than 10 mph and cloudiness equals to 0 
narrowed_city = city_weather.loc[(city_weather['Max Temp'] >=70) & (city_weather['Max Temp'] <= 80)
                                 & (city_weather['Windspeed'] <= 10) & (city_weather['Cloudiness'] ==0), :].dropna()
narrowed_city.reset_index(inplace=True)
del narrowed_city['index']
narrowed_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,miram shah,33.0086,70.0697,70.27,22.0,0.0,9.73,PK,1.612957e+09
1,chuy,-33.6971,-53.4616,72.05,80.0,0.0,7.40,UY,1.612957e+09
2,mirabad,30.4362,61.8383,77.27,17.0,0.0,5.99,AF,1.612957e+09
3,hervey bay,-25.2986,152.8535,73.99,73.0,0.0,3.00,AU,1.612957e+09
4,tahta,26.7693,31.5021,78.80,27.0,0.0,6.91,EG,1.612957e+09
5,alexandria,31.2156,29.9553,71.60,35.0,0.0,6.91,EG,1.612957e+09
6,santa vitoria do palmar,-33.5189,-53.3681,72.01,81.0,0.0,6.98,BR,1.612957e+09
7,vila velha,-20.3297,-40.2925,79.00,78.0,0.0,1.14,BR,1.612957e+09
8,sao joao da barra,-21.6403,-41.0511,75.20,83.0,0.0,6.91,BR,1.612957e+09
9,marsh harbour,26.5412,-77.0636,73.15,80.0,0.0,5.39,BS,1.612957e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df= narrowed_city.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,miram shah,PK,33.0086,70.0697,
1,chuy,UY,-33.6971,-53.4616,
2,mirabad,AF,30.4362,61.8383,
3,hervey bay,AU,-25.2986,152.8535,
4,tahta,EG,26.7693,31.5021,
5,alexandria,EG,31.2156,29.9553,
6,santa vitoria do palmar,BR,-33.5189,-53.3681,
7,vila velha,BR,-20.3297,-40.2925,
8,sao joao da barra,BR,-21.6403,-41.0511,
9,marsh harbour,BS,26.5412,-77.0636,


In [7]:
# set up a parameters dictionary
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from hotel_df
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    # add location to parameters dictionary
    params['location'] = f'{lat},{lng}'
    
    # assemble url and make API request
    print(f'Retrieving Results for Index {index}: {city_name}.')

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f'Closest hotel in {city_name} is {results[0]["name"]}.')
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print(f'Hotel not found. Skipping...')
    print(f'--------------')
print(f'---------------------\nSearch Complete\n--------------------')



Retrieving Results for Index 0: miram shah.
Closest hotel in miram shah is Bravo.
--------------
Retrieving Results for Index 1: chuy.
Closest hotel in chuy is Turis Firper Hotel.
--------------
Retrieving Results for Index 2: mirabad.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 3: hervey bay.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 4: tahta.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 5: alexandria.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 6: santa vitoria do palmar.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 7: vila velha.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 8: sao joao da barra.
Closest hotel in sao joao da barra is OYO Pousada Mediterrâneo.
--------------
Retrieving Results for Index 9: marsh harbour.
Hotel not found. Skipping...
--------------
Retrieving Results for Index 10: taoudenni.
Hotel n

In [9]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,miram shah,PK,33.0086,70.0697,Bravo
1,chuy,UY,-33.6971,-53.4616,Turis Firper Hotel
2,mirabad,AF,30.4362,61.8383,
3,hervey bay,AU,-25.2986,152.8535,
4,tahta,EG,26.7693,31.5021,
5,alexandria,EG,31.2156,29.9553,
6,santa vitoria do palmar,BR,-33.5189,-53.3681,
7,vila velha,BR,-20.3297,-40.2925,
8,sao joao da barra,BR,-21.6403,-41.0511,OYO Pousada Mediterrâneo
9,marsh harbour,BS,26.5412,-77.0636,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))